## MSA 8040 Project III - Fall 2020
-- Aishat Olayiwola, Anitha Karunakaran, Patrick Silva, Weiting Yu

### Web Scraping Code

In [ ]:
#Importing the libraries
import csv
import time 
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys

import requests, re 
from bs4 import BeautifulSoup
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

from collections import Counter

import random
import sys

## Phase 1

### This step gathers all URLs from the main page without filtering the content of the transcript (All sectors)
There are 30 items per page with a total of 4391793 (assuming the data-id HTML element represents unique id number of that transcript) as the last data-id on Nov27 @ 10:07PM. Therefore, we'll have ~146,393.1 pages to scrape, where only unique transcripts are shown in each page. However, as of today, the last page with information is 6427 (information from Apr. 24, 2004, 9:34 AM).

**Notes:**
- a) We only want data for year 2019 and 2020 (beginning at p.1169 as of Dec 06 11:22PM).
- b) Not all transcripts are shown as some of the pages jump the data-if by 2 or more numbers.
- c) Some transcripts requires a subscription to be accessed.

In [ ]:
class MSA8040_SeekingAlphaBot:

    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.driver = webdriver.Firefox()

    def closeBrowser(self):
        self.browser.close()

    def login(self):
        #Visits the Website
        driver = self.driver
        driver.get("https://seekingalpha.com")
        time.sleep(2)
        
        #Click on Sign In Button and Proceeds with the Login
        login_button = driver.find_element_by_xpath("//button[@data-test-id='header-button-sign-in']")
        login_button.click()
        time.sleep(2)
        
        ##Login Parameters
        user_name_elem = driver.find_element_by_xpath("//input[@name='email']")
        user_name_elem.clear()
        user_name_elem.send_keys(self.username)
        passworword_elem = driver.find_element_by_xpath("//input[@name='password']")
        passworword_elem.clear()
        passworword_elem.send_keys(self.password)
        passworword_elem.send_keys(Keys.RETURN)
        time.sleep(2)
        
        #Sets the URL coming from phase 1
        i = 1
        data_set_phase1 = []
        
        while i <= 1169: 
       
            base_url="https://seekingalpha.com/earnings/earnings-call-transcripts/"

            time.sleep(2) 
            
            driver.get(base_url+str(i))
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, "lxml")
            all=soup.find_all("li",{"class":"list-group-item article"})

            for item in all:

                #enter dictionary for store the data
                d={}
                #--------------------------------------------------#
                d["Title"]=item.find("h3",{"class":"list-group-item-heading"}).text.replace("\n","")

                #--------------------------------------------------# 
                try:
                    d["Url"]="https://seekingalpha.com"+item.find("a")['href']

                except:
                    d["Url"]=None
                #--------------------------------------------------#              
                data_set_phase1.append(d)

            i += 1
            print(i)
            
        #--------------------------------------------------------------------------------------------#    
        df = pd.DataFrame(data_set_phase1)
            
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('new_urls.xlsx', engine='xlsxwriter')
            
        # Convert the dataframe to an XlsxWriter Excel object.
        df.to_excel(writer, sheet_name='Sheet1')
            
        # Close the Pandas Excel writer and output the Excel file.
        writer.save()            
            
Project3 = MSA8040_SeekingAlphaBot("patricks.s@hotmail.com","msa80402020")
Project3.login()

In [ ]:
df_data_set_phase1 = pd.read_excel(r'C:\Users\patri\Documents\GSU\MSA 8040 - Data Management\Project 03\new_urls.xlsx')

df_phase1=pd.DataFrame(df_data_set_phase1[["Title","Url"]]).drop_duplicates()

display(df_phase1.iloc[0,1])
display(df_phase1.shape)

#Dropping Duplicates (if any)
df_url=pd.DataFrame(df_phase1["Url"]).drop_duplicates()
df_url.shape

## Phase 2

-- In this phase, we used the Bot to scrap the data from the URLs gathered from Phase 1. Each URL is a transcript and the result of the Phase 2 is a dataframe where each row is the content of the transcript.

-- Note that the process during Phase I has originated a dataframe with 17,190 unique URLs. Since running all URLs is computationally expensive, we randomly selected 2,000 URLs in order to get a more diverse dataset.

In [ ]:
#Get Sample
def get_sample(df, col_name, n=100, seed=42):
    """Get a sample from a column of a dataframe.
    
    It drops any numpy.nan entries before sampling. The sampling
    is performed without replacement.
    
    Example of numpydoc for those who haven't seen yet.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Source dataframe.
    col_name : str
        Name of the column to be sampled.
    n : int
        Sample size. Default is 100.
    seed : int
        Random seed. Default is 42.
    
    Returns
    -------
    pandas.Series
        Sample of size n from dataframe's column.
    """
    np.random.seed(seed)
    
    random_idx = np.random.choice(df[col_name].dropna().index, size=n, replace=False)
    
    return df.loc[random_idx, col_name]

In [ ]:
df_data_set_phase1 = pd.read_excel(r'C:\Users\patri\Documents\GSU\MSA 8040 - Data Management\Project 03\new_urls.xlsx')

df_phase1=pd.DataFrame(df_data_set_phase1[["Title","Url"]]).drop_duplicates()

display(df_phase1.iloc[0,1])
display(df_phase1.shape)

#Dropping Duplicates
df_url=pd.DataFrame(df_phase1["Url"]).drop_duplicates()
df_url.shape

In [ ]:
sample_url = pd.DataFrame(get_sample(df_url,"Url",2000))

In [ ]:
class MSA8040_SeekingAlphaBot:

    def __init__(self, username, password):
        self.username = username
        self.password = password
        self.driver = webdriver.Firefox()

    def closeBrowser(self):
        self.browser.close()

    def login(self):
        #Visits the Website
        driver = self.driver
        driver.get("https://seekingalpha.com")
        time.sleep(2)
        
        #Click on Sign In Button and Proceeds with the Login
        login_button = driver.find_element_by_xpath("//button[@data-test-id='header-button-sign-in']")
        login_button.click()
        time.sleep(2)
        
        ##Login Parameters
        user_name_elem = driver.find_element_by_xpath("//input[@name='email']")
        user_name_elem.clear()
        user_name_elem.send_keys(self.username)
        passworword_elem = driver.find_element_by_xpath("//input[@name='password']")
        passworword_elem.clear()
        passworword_elem.send_keys(self.password)
        passworword_elem.send_keys(Keys.RETURN)
        time.sleep(2) 
        
        #============================================== PHASE 2 ===============================================#
        i = 0
        base_url = ""
        transcript_title = []
        transcript_content = []
         
        while i <= len(sample_url)-1: 
            
            base_url = str(sample_url.iloc[i,0])
            
            time.sleep(2) 
            driver.get(base_url)
            time.sleep(2)
            html = driver.page_source
            soup = BeautifulSoup(html, "lxml")
            all1=soup.find_all("div",{"class":"sa-art-hd"}) #title
            all2=soup.find_all("div",{"class":"sa-art article-width"}) #Content
            
            #------------------------------------ TITLE --------------------------------------------------------#
            for item in all1:

                # enter dictionary for store the data
                d1 = {}
                try:
                    d1["Title"] = str(item.find("h1")).split(">")[1].split("<")[0]
                    time.sleep(2)

                except Exception:
                    continue

                transcript_title.append(d1)

            #------------------------------------ CONTENT --------------------------------------------------------#
            for item in all2:

                # enter dictionary for store the data
                d2 = {}
                try:
                    d2["Content"] = item.text
                    time.sleep(2)

                except Exception:
                    continue

                transcript_content.append(d2)

            i += 1
            print(i)
         #-----------------------------------------------------------------------------------------------------#   
        df = pd.concat([pd.DataFrame(transcript_title), pd.DataFrame(transcript_content)], axis=1)
            
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter('df_transcripts.xlsx', engine='xlsxwriter')
            
        # Convert the dataframe to an XlsxWriter Excel object.
        df.to_excel(writer, sheet_name='Sheet1')
            
        # Close the Pandas Excel writer and output the Excel file.
        writer.save()
        
Project3 = MSA8040_SeekingAlphaBot("patricks.s@hotmail.com","msa80402020")
Project3.login()       

## Phase 3

-- The last thing we need to do is to use techniques to parse the data from Phase 2 (content columns) and build the final dataset with the desired attributes.

-- After Phase 2, we noticed that due to subscription requirements of the website, not every 2,000 transcripts have been fully scraped. In addition, this step will involve parsing the data in order to get a structured data by spliting the rows into columns and then understanding the logic to create each attribute separately. So, the size of the df_transcripts file has been reduced to 286 (df_transcripts_selection.xlsx) after further review and then reduced to 235 transcripts as a final dataset (final_dataset.csv).

In [ ]:
df = pd.read_excel('df_transcripts_selection.xlsx')
prow = pd.DataFrame(columns = ['Ticker','Company','Title','Date','Time','Speech','Participant name'])

def run(context):
    df_list=context.split("\n")
    df_list =list(filter(lambda a: a != '', df_list ))
    def listToString(s):  

        # initialize an empty string 
        str1 = " " 
        # return string   
        return (str1.join(s)) 
    def find_company(df):
        Company=df_list[0].split('(')[0]
        Ticker=df_list[0].split('(')[1].split(')')[0].split(':')[1]
        a=df_list[0].split(')')[1].split(' ')[:-7]
        Conference=listToString(a)
        Month=df_list[0].split(')')[1].split(' ')[-7]
        Date=df_list[0].split(')')[1].split(' ')[-6]
        Year=df_list[0].split(')')[1].split(' ')[-5]
        Time=df_list[0].split(')')[1].split(' ')[-3]
        AM_PM=df_list[0].split(')')[1].split(' ')[-2]
        Time_Zone=df_list[0].split(')')[1].split(' ')[-1]
        df_Company= pd.DataFrame()
        df_Company['Ticker']=[Ticker]
        df_Company['Company']=[Company]
        df_Company['Conference']=[Conference]
        df_Company['Month']=[Month]
        df_Company['Date']=[Date]
        df_Company['Year']=[Year]
        df_Company['Time']=[Time]
        df_Company['AM_PM']=[AM_PM]
        df_Company['Time_Zone']=[Time_Zone]
        return df_Company
    find_company=find_company(df)
    def find_Company_Participants(df):
        i=0
        Company_Participants_list=[]
        Conference_Call_Participants_list=[]
        while i>=0:
            i=i+1
            if df_list[i]=='Conference Call Participants':
                break
            Company_Participants_list.append(i)   
        while i==i:
            i=i+1
            if len(df_list[i])>100:
                break
            Conference_Call_Participants_list.append(i)
        Company_Participants_list=Company_Participants_list[1:]
        Conference_Call_Participants_list=Conference_Call_Participants_list[:-1]
        Company_Participants_name=[]
        Company_Participants_type=[]
        for i in Company_Participants_list:
            name_title=df_list[i]
            Company_Participants_name.append((name_title.split('-')[0]))
            Company_Participants_type.append(name_title.split('-')[1])
        df_Company_Participants= pd.DataFrame( {'Participant name': Company_Participants_name, 'Company_Participants_type': Company_Participants_type})
        return df_Company_Participants
    find_Company_Participants=find_Company_Participants(df)
    def find_Conference_Call_Participants(df):
        i=0
        Company_Participants_list=[]
        Conference_Call_Participants_list=[]
        while i>=0:
            i=i+1
            if df_list[i]=='Conference Call Participants':
                break
            Company_Participants_list.append(i)   
        while i==i:
            i=i+1
            if len(df_list[i])>100:
                break
            Conference_Call_Participants_list.append(i)
        Company_Participants_list=Company_Participants_list[1:]
        Conference_Call_Participants_list=Conference_Call_Participants_list[:-1]
        Conference_Call_Participants_name=[]
        Conference_Call_Participants_type=[]
        for i in Conference_Call_Participants_list:
            name_title=df_list[i]
            Conference_Call_Participants_name.append((name_title.split('-')[0]))
            Conference_Call_Participants_type.append(name_title.split('-')[1])
        df_Conference_Call_Participants= pd.DataFrame( {'Participant name': Conference_Call_Participants_name, 'Participant_Organization': Conference_Call_Participants_type})
        return df_Conference_Call_Participants
    find_Conference_Call_Participants=find_Conference_Call_Participants(df)

    df_new=pd.DataFrame({'Speech': df_list,'Participant name':'','Section':'presentation'})
    #####################name
    name_index=[]
    for i in find_Company_Participants['Participant name']:
        name=(str(i)[:-1])
        a=[i for i,x in enumerate(df_list) if x==name]
        name_index=name_index+a
    for i in find_Conference_Call_Participants['Participant name']:
        name=(str(i)[:-1])
        a=[i for i,x in enumerate(df_list) if x==name]
        name_index=name_index+a
    Operator=[i for i,x in enumerate(df_list) if x=='Operator']
    name_index=name_index+Operator
    name_index.sort()
    for i in name_index:
        df_new.iloc[i:,1]=df_new.iloc[i,0]
   ####################QA
    QA_index=[i for i,x in enumerate(df_list) if x=='Question-and-Answer Session']
    for i in QA_index:
        df_new.iloc[i:,2]='Question-and-Answer Session'
    
        
        
        
    df_new=df_new[df_new['Speech'].str.len()>110]
    df_new['Ticker']=find_company['Ticker'][0]
    df_new['Company']=find_company['Company'][0]
    
    
    clean_2=list(filter(lambda a: a != '',df_list[0].split(')')[1].split(' ')))
    df_new['Title']=listToString(clean_2[:-7])
    
    
    clean_1 =list(filter(lambda a: a != '', df_list[0].split(')')[1].split(' ') ))
    df_new['Date']=listToString(clean_1[-6:-3])
    df_new['Time']=listToString(df_list[0].split(')')[1].split(' ')[-3:])    
    df_new['Participant name']=df_new['Participant name']+" "
    
    df_new=df_new[['Ticker','Company','Title','Date','Time','Section','Speech','Participant name']]
    #####add Participant_Organization/Company_Participants_type
    df_new=pd.merge(df_new,find_Company_Participants, how='left', left_on=['Participant name'], right_on=['Participant name'])
    df_new=pd.merge(df_new,find_Conference_Call_Participants, how='left', left_on=['Participant name'], right_on=['Participant name'])
    #prow=prow.append(df_new)
    
    return df_new

In [ ]:
i=0
while i<len(df['Content']):
    i=i+1
    try:
        prow =prow.append(run(df['Content'][i-1]))
    except IndexError:
        pass

In [ ]:
prow['Company_Participants_type']=prow['Company_Participants_type'].replace(np.nan, 'attendee')
display(prow)

prow.to_csv("final_dataset.csv",index=False)